# Set up

In [1]:
! pip install pymongo[srv,tls]
! pip install dnspython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.8 MB/s eta 0:00:00


In [2]:
from pymongo.mongo_client import MongoClient
from datetime import date
from contextlib import nullcontext
import pandas as pd

uri = "mongodb+srv://yanaput:<password>@workshop-bakery.qtngygm.mongodb.net/?retryWrites=true&w=majority&appName=Workshop-Bakery"

client = MongoClient(uri)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

db = client.Bakery



Pinged your deployment. You successfully connected to MongoDB!


In [3]:
#@title Insert Cake Info

Name = "exampleCake" #@param {type:"string"}
ShortDescription = "example" #@param {type:"string"}
Description = "example" #@param {type:"string"}
Image = "exampleURL" #@param {type:"string"}
Ingredient = "exampleIngre0, exampleIngre1, exampleIngre2, exampleIngre3" #@param {type:"string"}
Recipe = "example text"#@param {type:"string"}
Stock =  33 #@param {type:"slider", min:1, max:80, step:1}
Price = 10 #@param {type: "number"}



In [6]:
cake = {  'name' : Name ,
          'shortDescription' : ShortDescription,
          'description' : Description,
          'image' : Image,
          'ingredients' : Ingredient.split(","),
          'recipe' : Recipe,
          'stock' : Stock,
          'price' : Price
       }

if len(list(db.Cakes.find({'name' : cake['name']}))) > 0 :
  print("The cake already existed replacing instead")
  print(db.Cakes.replace_one({'name': cake['name']}, cake))
else :
  print(pd.db.Cakes.insert_one(cake))


The cake already existed replacing instead
UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff000000000000065e'), 'opTime': {'ts': Timestamp(1716196114, 39), 't': 1630}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1716196114, 39), 'signature': {'hash': b'\x1auQ\xb8\x95\xcdjb\xd9\xa7^\xe9\x11S\x11\x1f\xc7\xa0\xe0g', 'keyId': 7329897054002479105}}, 'operationTime': Timestamp(1716196114, 39), 'updatedExisting': True}, acknowledged=True)


In [7]:
#@title Insert Customer Info

#@markdown Name
Name = "Mark" #@param {type:"string"}
LastName = "Kuprasertwong" #@param {type:"string"}

#@markdown Address
Address = "example" #@param {type:"string"}
Country = "Thailand" #@param {type:"string"}
City = "example" #@param {type:"string"}
Province = "example" #@param {type :"string"}
PostalCode = "73170" #@param {type:"string"}

#@markdown Contact information
Email = "example@email.com" #@param {type:"string"}
Phone = "0850000000" #@param {type:"string"}


In [11]:
customerInfo = {
    'name' : Name,
    'lastName' : LastName,
    'address' : Address,
    'country' : Country,
    'city' : City,
    'province' : Province,
    'postalCode' : PostalCode,
    'email' : Email,
    'phone' : Phone }

if len(list(db.addressbook.find({'name' : customerInfo['name'],
                                'lastName' : customerInfo['lastName']}))) > 0 :
  print("The info already existed replacing instead")
  print(db.addressbook.replace_one({'name' : customerInfo['name'],
                                'lastName' : customerInfo['lastName']}, customerInfo))
else :
  print("Added info")
  print(db.addressbook.insert_one(customerInfo))

The info already existed replacing instead
UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff000000000000065e'), 'opTime': {'ts': Timestamp(1716196179, 39), 't': 1630}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1716196179, 40), 'signature': {'hash': b'\xae\x0cD/\xef\xfe\x8b\t\x05!\xf2\xfb\xe6\x0eM\x1dS\xb5S\x9c', 'keyId': 7329897054002479105}}, 'operationTime': Timestamp(1716196179, 39), 'updatedExisting': True}, acknowledged=True)


In [ ]:
#@title Insert Order

CustomerName = "Mark" #@param {type:"string"}
LastName = "Kuprasertwong" #@param {type:"string"}
Cake = "Cheese Cake"#@param {type:"string"}
Quantity = 2 #@param {type:"integer"}
OrderDate = '2024-05-16' #@param {type:"date"}
RetrieveDate = '2024-05-16' #@param {type:"date"}
RetrieveOption = "Pickup" #@param ["Pickup", "Delivery"]


In [ ]:
orderedCake = db.Cakes.find_one({'name': Cake})

order = {
    'customerName' : CustomerName,
    'lastName' : LastName,
    'cake' : Cake,
    'quantity' : Quantity,
    'subTotal' : orderedCake['price'] * Quantity,
    'orderDate' : OrderDate ,
    'retrieveDate' : RetrieveDate,
    'retrieveOption' : RetrieveOption
}



if db.Cakes.find_one({'name': order['cake']}) is not None :
  updatedCake = db.Cakes.find_one_and_update({'name': order['cake']},
                      {'$inc': {'stock': -Quantity if orderedCake['stock'] > 0 and orderedCake['stock'] > Quantity else 0 }},
                      return_document=True)
  if orderedCake['stock'] < 0 :
     print(f"{order['cake']} is out of stock")
  elif orderedCake['stock'] < Quantity :
    print(f"{order['cake']} quantity is not enough")
  else:
    print(db.order.insert_one(order))
    print("Stock updated:", updatedCake)
else :
  print(f"error no {order['cake']} in the shop")



InsertOneResult(ObjectId('6648b4df07e347e5fd7dba22'), acknowledged=True)
Stock updated: {'_id': ObjectId('64759e4c3beb40594cb859ed'), 'name': 'Cheese Cake', 'shortDescription': 'Cheesecake is a sweet dessert consisting of one or more layers. The main, and thickest, layer consists of a mixture of a soft, fresh cheese (typically cottage cheese, cream cheese or ricotta), eggs, and sugar. ', 'description': 'Cheesecake is a sweet dessert consisting of one or more layers. The main, and thickest, layer consists of a mixture of a soft, fresh cheese (typically cottage cheese, cream cheese or ricotta), eggs, and sugar. If there is a bottom layer, it most often consists of a crust or base made from crushed cookies (or digestive biscuits), graham crackers, pastry, or sometimes sponge cake.[1] Cheesecake may be baked or unbaked (and is usually refrigerated).', 'image': 'https://sallysbakingaddiction.com/wp-content/uploads/2018/05/perfect-cheesecake-recipe.jpg', 'ingredients': ['graham cracker crumb

In [24]:
#@title Search cake by name

SearchCake = "Chocolate Cake" #@param {type:"string"}

#<--------------------------------------------------------------------------->

if SearchCake is not None and SearchCake != "":
  result = db.Cakes.find({'name' : SearchCake})
  if result is not None:
    print("Cake info : ")
    cakeTable = pd.DataFrame(list(result))
  else:
    print(f"no {SearchCake} cake in database")
cakeTable

Cake info : 


,_id,name,shortDescription,description,image,ingredients,recipe,stock,price
0,64759d513beb40594cb859e9,Chocolate Cake,Chocolate cake is a cake flavored with melted ...,Chocolate cake is made with chocolate; it can ...,https://addapinch.com/wp-content/uploads/2020/...,"[flour, sugar, cocoa powder]",Preheat your oven to the specified temperature...,13,20


In [33]:
#@title Search customer info by [name lastname]
SearchCustomerInfo = "Mark Kuprasertwong" #@param {type : "string"}
#<--------------------------------------------------------------------------->
#Search customer info by [name lastname] in database
if SearchCustomerInfo is not None and SearchCustomerInfo != "":
  customerNameLastname = SearchCustomerInfo.split()
  if len(customerNameLastname) > 1 :
    result = db.addressbook.find({'name' : customerNameLastname[0],
                                'lastName' : customerNameLastname[1]})
    if result is not None:
      print("Cutomer info : ")
      customerInfoTable = pd.DataFrame(list(result))
    else:
      print(f"no {SearchCustomerInfo} customer in database")
  else:
    print("Customer info's format is incorrect")

customerInfoTable


Cutomer info : 


,_id,name,lastName,address,country,city,province,postalCode,email,phone
0,6648b5b407e347e5fd7dba24,Mark,Kuprasertwong,example,Thailand,example,example,73170,example@email.com,0850000000


In [34]:
#@title Search ordered by cake
SearchCakeOrder = "Chocolate Cake" #@param {type : "string"}

#<--------------------------------------------------------------------------->

if SearchCakeOrder is not None and SearchCakeOrder != "":
  result = db.order.find({'cake' : SearchCakeOrder})
  if result is not None:
    print("Cakes in orders :")
    CakeOrderTable = pd.DataFrame(list(result))
  else:
    print(f"no {SearchCakeOrder} cake in order")
CakeOrderTable

Cakes in orders :


,_id,customerName,lastName,cake,quantity,subTotal,orderDate,retrieveDate,retrieveOption
0,6648b15007e347e5fd7dba1e,exampleName,exampleLastName,Chocolate Cake,2,40,2024-05-16,2024-05-16,Delivery
1,6648b1a907e347e5fd7dba1f,exampleName,exampleLastName,Chocolate Cake,2,40,2024-05-16,2024-05-16,Delivery


In [32]:
#@title Search order by [name lastname]
SearchCustomerOrder = "exampleName exampleLastName" #@param {type : "string"}

#<--------------------------------------------------------------------------->


if SearchCustomerOrder is not None and SearchCustomerOrder != "":
  customerNameLastname = SearchCustomerOrder.split()
  if len(customerNameLastname) > 1 :
    result = db.order.find({'customerName' : customerNameLastname[0],
                                'lastName' : customerNameLastname[1]})
    if result is not None:
      print(f"Customer [{SearchCustomerOrder}]'s order :")
      customerInfoTable = pd.DataFrame(list(result))
      # for item in list(result):
      #   print(item)
    else:
      print(f"no [{SearchCustomerOrder}]'s order in database")
  else:
    print("Customer info's format is incorrect")
customerInfoTable

Customer [exampleName exampleLastName]'s order :


,_id,customerName,lastName,cake,quantity,subTotal,orderDate,retrieveDate,retrieveOption
0,6648b15007e347e5fd7dba1e,exampleName,exampleLastName,Chocolate Cake,2,40,2024-05-16,2024-05-16,Delivery
1,6648b1a907e347e5fd7dba1f,exampleName,exampleLastName,Chocolate Cake,2,40,2024-05-16,2024-05-16,Delivery
